In [7]:
# Instal library yang dibutuhkan
!pip install replicate pandas
!pip install langchain_community

# Impor semua library yang diperlukan di awal
import pandas as pd
import replicate
from langchain_community.llms import Replicate
import re
import os
import time
from google.colab import drive
from google.colab import userdata

In [8]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# =======================================================================
# BAGIAN 1: KONEKSI DAN KONFIGURASI
# =======================================================================

# Mengatur API Token Replicate dari Colab Secrets untuk keamanan
try:
    os.environ['REPLICATE_API_TOKEN'] = userdata.get('api')
    print("✅ Token API Replicate berhasil dimuat.")
except Exception as e:
    print("⚠️ Gagal memuat token API. Pastikan Anda sudah membuat Secret dengan nama 'REPLICATE_API_TOKEN' dan mengaktifkan 'Notebook access'.")

# --- KONFIGURASI PROYEK
PATH_INPUT_FILE = '/content/drive/MyDrive/Colab Notebooks/youtube_comments_simple.csv'
PATH_OUTPUT_FILE = '/content/drive/MyDrive/Colab Notebooks/hasil_klasifikasi.csv'
NAMA_KOLOM_KOMENTAR = 'textDisplay'
JUMLAH_SAMPEL_DATA = 10

print("✅ Bagian 1: Koneksi dan Konfigurasi selesai.")

# =======================================================================
# BAGIAN 2: DEFINISI FUNGSI
# =======================================================================

def clean_text(text: str) -> str:
    """Fungsi untuk membersihkan teks dari tag HTML, URL, mentions, dan karakter non-alfanumerik."""
    if not isinstance(text, str):
        return ""
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'@[A-Za-z0-9_]+', ' ', text)
    text = re.sub(r'https?://\S+', ' ', text)
    text = re.sub(r'[^A-Za-z0-9 ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def ask_replicate(prompt: str) -> str | None:
    """Fungsi untuk mengirim prompt ke model Granite di Replicate."""
    model_id = "ibm-granite/granite-3.3-8b-instruct"

    try:
        output = replicate.run(
            model_id,
            input={
                "prompt": prompt,   # gunakan "prompt", bukan "chat_history"
                "temperature": 0.1,
                "max_new_tokens": 128
            }
        )
        if output:
            return "".join(output).strip()
        else:
            return None
    except Exception as e:
        print(f"  -> Terjadi Error pada API Replicate: {e}")
        return None


print("✅ Bagian 2: Definisi fungsi selesai.")

# =======================================================================
# BAGIAN 3: EKSEKUSI UTAMA (MEMUAT, MEMPROSES, MENYIMPAN)
# =======================================================================

print("\n🚀 Memulai Eksekusi Utama Proyek...")

# Memuat dataset utama dari path yang sudah ditentukan
try:
    df = pd.read_csv(PATH_INPUT_FILE)
    print(f"✅ Dataset berhasil dimuat dari Google Drive. Total baris: {len(df)}")
except FileNotFoundError:
    print(f"❌ FATAL ERROR: File dataset tidak ditemukan di '{PATH_INPUT_FILE}'. Harap periksa kembali path file Anda.")
    df = None

# Proses hanya akan berjalan jika dataset berhasil dimuat
if df is not None:
    # 1. KLASIFIKASI KOMENTAR
    df_sample = df.head(JUMLAH_SAMPEL_DATA).copy()
    print(f"⚙️ Mengambil {len(df_sample)} baris pertama untuk diproses sebagai sampel.")

    hasil_klasifikasi = []

    print("\n⏳ Memulai proses klasifikasi komentar...")
    for index, row in df_sample.iterrows():
        comment_text = row[NAMA_KOLOM_KOMENTAR]
        cleaned_comment = clean_text(comment_text)

        print(f"  -> Memproses komentar ke-{index + 1} dari {len(df_sample)}...")

        if not cleaned_comment:
            print("     -> Komentar kosong setelah dibersihkan, dilewati.")
            hasil_klasifikasi.append("Komentar Kosong")
            continue

        classification_prompt = f"""
Klasifikasikan komentar YouTube berikut ke dalam salah satu kategori ini: Pro-Streetfeeding, Kontra-Streetfeeding, Edukasi, Pertanyaan, atau Lainnya. Berikan hanya nama kategorinya saja tanpa penjelasan tambahan.

Komentar: "{cleaned_comment}"

Kategori:
"""
        category = ask_replicate(classification_prompt)
        hasil_klasifikasi.append(category if category else "Gagal Diproses")
        time.sleep(1)

    df_sample['kategori'] = hasil_klasifikasi
    print("✅ Proses klasifikasi selesai.")

    df_sample.to_csv(PATH_OUTPUT_FILE, index=False)
    print(f"💾 Hasil analisis telah berhasil disimpan ke Google Drive di: '{PATH_OUTPUT_FILE}'")

    print("\nBerikut adalah 5 baris pertama dari hasil analisis Anda:")
    print(df_sample.head())

    # 2. RANGKUMAN PER KATEGORI
    print("\n⏳ Memulai proses rangkuman per kategori...")
    kategori_unik = df_sample['kategori'].unique()

    for kategori in kategori_unik:
        if kategori in ["Gagal Diproses", "Komentar Kosong"]:
            continue

        print(f"  -> Merangkum argumen untuk kategori: '{kategori}'")
        komentar_grup = ". ".join(df_sample[df_sample['kategori'] == kategori][NAMA_KOLOM_KOMENTAR].astype(str))

        summarization_prompt = f"""
Berdasarkan kumpulan komentar berikut yang semuanya masuk dalam kategori '{kategori}', rangkum poin dan argumen utamanya dalam satu atau dua kalimat singkat.

Komentar: "{komentar_grup}"

Rangkuman:
"""
        summary = ask_replicate(summarization_prompt)

        if summary:
            print(f"     Rangkuman: {summary}\n")
        else:
            print(f"     Gagal merangkum kategori '{kategori}'.\n")
        time.sleep(1)

    print("✅ Proses rangkuman selesai.")
    print("\n🎉 SELURUH PROSES TELAH SELESAI 🎉")

✅ Token API Replicate berhasil dimuat.
✅ Bagian 1: Koneksi dan Konfigurasi selesai.
✅ Bagian 2: Definisi fungsi selesai.

🚀 Memulai Eksekusi Utama Proyek...
✅ Dataset berhasil dimuat dari Google Drive. Total baris: 9166
⚙️ Mengambil 10 baris pertama untuk diproses sebagai sampel.

⏳ Memulai proses klasifikasi komentar...
  -> Memproses komentar ke-1 dari 10...
  -> Memproses komentar ke-2 dari 10...
  -> Memproses komentar ke-3 dari 10...
  -> Memproses komentar ke-4 dari 10...
  -> Memproses komentar ke-5 dari 10...
  -> Memproses komentar ke-6 dari 10...
  -> Memproses komentar ke-7 dari 10...
  -> Memproses komentar ke-8 dari 10...
  -> Memproses komentar ke-9 dari 10...
  -> Memproses komentar ke-10 dari 10...
✅ Proses klasifikasi selesai.
💾 Hasil analisis telah berhasil disimpan ke Google Drive di: '/content/drive/MyDrive/Colab Notebooks/hasil_klasifikasi.csv'

Berikut adalah 5 baris pertama dari hasil analisis Anda:
      video_id       authorDisplayName  \
0  3YeYUoybimM        